In [1]:
# EOMs
%reset
#import libraries
import sympy as sym
import numpy as np
import pickle as pickle

sym.init_printing()
from IPython.display import display #for pretty printing

# Symbols --------------------------------------------------
legs = ['A']
links = ['ull','lll','url','lrl','foot']
bases = ['body','boom']

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#Parameters
masses = {}
lengths = {}
inertias = {}

LL = sym.symbols('LL')
g = sym.symbols('g')
foot_angle = 150*(np.pi/180)

for leg in legs:
    for link in links:   
        masses.update({(link,leg): sym.symbols('m_{0}'.format(link+leg))})
        lengths.update({(link,leg): sym.symbols('l_{0}'.format(link+leg))})
        inertias.update({(link,leg): sym.symbols('In_{0}'.format(link+leg))})

for base in bases:
    masses.update({(base): sym.symbols('m_{0}'.format(base))})
    lengths.update({(base): sym.symbols('l_{0}'.format(base))})
    inertias.update({(base): sym.symbols('In_{0}'.format(base))})

#gen coordinates
x_body,y_body,th_body = sym.symbols(['x_body','y_body','theta_{0}'.format('body')])
dx_body,dy_body,dth_body = sym.symbols(['\dot{\\x}_{body}','\dot{\\y}_{body}','\dot{\\theta}_{body}'])
ddx_body,ddy_body,ddth_body = sym.symbols(['\ddot{\\x}_{body}','\ddot{\\y}_{body}','\ddot{\\theta}_{body}'])

th = {}
dth = {}
ddth = {}

for leg in legs:
    for link in links:
        if 'foot' in link:
            pass
        else:
            th.update({(link,leg): sym.symbols('theta_{0}'.format(link+leg))}) 
            dth.update({(link,leg): sym.symbols('\dot{{\\theta}}_{{{0}}}'.format(link+leg))})
            ddth.update({(link,leg): sym.symbols('\ddot{{\\theta}}_{{{0}}}'.format(link+leg))})

In [3]:
q = sym.Matrix([x_body]+[y_body]+[th_body]+[th[link,leg] for leg in legs for link in links if 'foot' not in link])
dq = sym.Matrix([dx_body]+[dy_body]+[dth_body]+[dth[link,leg] for leg in legs for link in links if 'foot' not in link])
ddq = sym.Matrix([ddx_body]+[ddy_body]+[ddth_body]+[ddth[link,leg] for leg in legs for link in links if 'foot' not in link])

In [4]:
q

⎡x_body⎤
⎢      ⎥
⎢y_body⎥
⎢      ⎥
⎢θ_body⎥
⎢      ⎥
⎢θ_ullA⎥
⎢      ⎥
⎢θ_lllA⎥
⎢      ⎥
⎢θ_urlA⎥
⎢      ⎥
⎣θ_lrlA⎦

In [5]:
dq

⎡  \dot{\x}_{body}  ⎤
⎢                   ⎥
⎢  \dot{\y}_{body}  ⎥
⎢                   ⎥
⎢\dot{\theta}_{body}⎥
⎢                   ⎥
⎢\dot{\theta}_{ullA}⎥
⎢                   ⎥
⎢\dot{\theta}_{lllA}⎥
⎢                   ⎥
⎢\dot{\theta}_{urlA}⎥
⎢                   ⎥
⎣\dot{\theta}_{lrlA}⎦

In [6]:
ddq

⎡  \ddot{\x}_{body}  ⎤
⎢                    ⎥
⎢  \ddot{\y}_{body}  ⎥
⎢                    ⎥
⎢\ddot{\theta}_{body}⎥
⎢                    ⎥
⎢\ddot{\theta}_{ullA}⎥
⎢                    ⎥
⎢\ddot{\theta}_{lllA}⎥
⎢                    ⎥
⎢\ddot{\theta}_{urlA}⎥
⎢                    ⎥
⎣\ddot{\theta}_{lrlA}⎦

In [7]:
''' Setup positions (to COM) and velocities'''

height = 2/3

r_bases = {}

# position to COM of each link - where the mass is approximated to be
r_bases.update({'body': sym.Matrix([[x_body,
                                     y_body]]).transpose()})

r_bases.update({'boom': sym.Matrix([[x_body*height,
                                     y_body*height]]).transpose()})

# define absolute angles
abs_angles = {}
r_links = {}


for leg in legs:
    
    abs_angles.update({('LU',leg): th_body + th['ull',leg]})
    abs_angles.update({('LL',leg): th['lll',leg] - (np.pi - abs_angles['LU',leg])})
    abs_angles.update({('RU',leg): th_body + th['url',leg]})
    abs_angles.update({('RL',leg): th['lrl',leg] - (np.pi - abs_angles['RU',leg]) }) 
    abs_angles.update({('foot',leg): foot_angle - abs_angles['RL',leg]})
    
    r_links.update({('ull',leg): sym.Matrix([[x_body-LL/2*sym.cos(th_body)-lengths['ull',leg]/2*sym.cos(abs_angles['LU',leg]),
                                              y_body-LL/2*sym.sin(th_body)-lengths['ull',leg]/2*sym.sin(abs_angles['LU',leg])]]).transpose()})
    
    r_links.update({('lll',leg): sym.Matrix([[r_links['ull',leg][0]-0.5*lengths['ull',leg]*sym.cos(abs_angles['LU',leg])+lengths['lll',leg]/2*sym.cos(np.pi-abs_angles['LL',leg]),
                                              r_links['ull',leg][1]-0.5*lengths['ull',leg]*sym.sin(abs_angles['LU',leg])-lengths['lll',leg]/2*sym.sin(np.pi-abs_angles['LL',leg])]]).transpose()})
    
    r_links.update({('url',leg): sym.Matrix([[x_body+LL/2*sym.cos(th_body)+lengths['url',leg]/2*sym.cos(np.pi-abs_angles['RU',leg]),
                                              y_body+LL/2*sym.sin(th_body)-lengths['url',leg]/2*sym.sin(np.pi-abs_angles['RU',leg])]]).transpose()})
    
    r_links.update({('lrl',leg): sym.Matrix([[r_links['url',leg][0]+0.5*lengths['url',leg]*sym.cos(np.pi-abs_angles['RU',leg])-lengths['lrl',leg]/2*sym.cos(abs_angles['RL',leg]),
                                              r_links['url',leg][1]-0.5*lengths['url',leg]*sym.sin(np.pi-abs_angles['RU',leg])-lengths['lrl',leg]/2*sym.sin(abs_angles['RL',leg])]]).transpose()})
    
    r_links.update({('foot',leg): sym.Matrix([[r_links['lrl',leg][0]-0.5*lengths['lrl',leg]*sym.cos(abs_angles['RL',leg])+lengths['foot',leg]/2*sym.cos(abs_angles['foot',leg]),
                                               r_links['lrl',leg][1]-0.5*lengths['lrl',leg]*sym.sin(abs_angles['RL',leg])-lengths['foot',leg]/2*sym.sin(abs_angles['foot',leg])]]).transpose()})

In [8]:
dr_bases = {}
dr_links = {}
w_links = {}
w_bases = {}

w_bases.update({'body': dth_body})

for base in bases:
    dr_bases.update({base: r_bases[base].jacobian(q)*dq})

for leg in legs:
    
    w_links.update({('ull',leg): dth['ull',leg] + dth_body})
    w_links.update({('lll',leg): dth['ull',leg] + dth['lll',leg] + dth_body})
    w_links.update({('url',leg): dth['url',leg] + dth_body})
    w_links.update({('lrl',leg): dth['url',leg] + dth['lrl',leg] + dth_body})
    ''' ****** '''
    w_links.update({('foot',leg): dth['url',leg] + dth['lrl',leg] + dth_body})
    
    for link in links:
        dr_links.update({(link,leg): r_links[link,leg].jacobian(q)*dq})
    

In [9]:
T_bases = {}
T_links = {}
V_bases = {}
V_links = {}

# Kinetic Energy
T_bases.update({'body': 0.5*sym.Matrix([[masses['body'],masses['body']]])*sym.matrix_multiply_elementwise(dr_bases['body'],dr_bases['body']) + sym.Matrix([[0.5*w_bases['body']*w_bases['body']*inertias['body']]])})
T_bases.update({'boom': 0.5*sym.Matrix([[masses['boom'],masses['boom']]])*sym.matrix_multiply_elementwise(dr_bases['boom'],dr_bases['boom'])})

# Potential Energy
for base in bases:
    V_bases.update({base: masses[base]*g*r_bases[base][1]})


for leg in legs:
    for link in links:
        T_links.update({(link,leg): 0.5*sym.Matrix([[masses[link,leg],masses[link,leg]]])*sym.matrix_multiply_elementwise(dr_links[link,leg],dr_links[link,leg]) + sym.Matrix([[0.5*w_links[link,leg]*w_links[link,leg]*inertias[link,leg]]])})
        V_links.update({(link,leg): masses[link,leg]*g*r_links[link,leg][1]})

Ttot = T_bases['body'] + T_bases['boom']
Vtot = V_bases['body'] + V_bases['boom']
    
for leg in legs:
    for link in links:
        Ttot = Ttot + T_links[link,leg] 
        Vtot = Vtot + V_links[link,leg] 
        
Ttot = Ttot[0]

In [10]:
# Manipulator Equation (M*ddq + C + G = Tau + Q)

# M Matrix
M = sym.zeros(len(q),len(q))

for i in range(len(q)):
    for j in range(len(q)):
        M[i,j] = sym.diff(sym.diff(Ttot,dq[i]),dq[j])

M = sym.simplify(M)

M_size = int(np.sqrt(len(M))) # assumes square matrix (len(M) = 4x4 = 16)

dM = sym.zeros(M_size,M_size)

for i in range(M_size):
    for j in range(M_size):
        dM[i,j] = sym.Matrix([M[i,j]]).jacobian(q)*dq

# C Matrix
C = dM*dq - (sym.Matrix([Ttot]).jacobian(q)).transpose() # 4x1 - 1x4^Transpose

# G Matrix
G = (sym.Matrix([Vtot]).jacobian(q)).transpose() # 4x1

tau = {}
lamda = {}
GRF = {}

connect_position = {}
foot_position = {}

Jbn = {}
Jfoot = {}
lamda_GRF = {}
lamda_connect = {}

Tau = sym.Matrix([[0],[0],[0]])

Q = sym.zeros(len(q),1)

for leg in legs:
    tau.update({('left',leg): sym.symbols('tau_left{0}'.format(leg))})
    tau.update({('right',leg): sym.symbols('tau_right{0}'.format(leg))})
    lamda.update({('x',leg): sym.symbols('lamda_x{0}'.format(leg))})
    lamda.update({('y',leg): sym.symbols('lamda_y{0}'.format(leg))})
    GRF.update({('x',leg): sym.symbols('GRF_x{0}'.format(leg))})
    GRF.update({('y',leg): sym.symbols('GRF_y{0}'.format(leg))})
    
    connect_position.update({('left',leg): sym.Matrix([[r_links['lll',leg][0]+0.5*lengths['lll',leg]*sym.cos(np.pi-abs_angles['LL',leg])],
                                                       [r_links['lll',leg][1]-0.5*lengths['lll',leg]*sym.sin(np.pi-abs_angles['LL',leg])]]).transpose()})
    
    connect_position.update({('right',leg): sym.Matrix([[r_links['lrl',leg][0]-0.5*lengths['lrl',leg]*sym.cos(abs_angles['RL',leg])],
                                                        [r_links['lrl',leg][1]-0.5*lengths['lrl',leg]*sym.sin(abs_angles['RL',leg])]]).transpose()})
    ''' ****** '''
    foot_position.update({(leg): sym.Matrix([[r_links['foot',leg][0]+0.5*lengths['foot',leg]*sym.cos(abs_angles['foot',leg])],
                                             [r_links['foot',leg][1]-0.5*lengths['foot',leg]*sym.sin(abs_angles['foot',leg])]]).transpose()})

    Jbn.update({('left',leg): connect_position['left',leg].jacobian(q)})                    
    Jbn.update({('right',leg): connect_position['right',leg].jacobian(q)})                    
    Jfoot.update({(leg): foot_position[leg].jacobian(q)})                    
    lamda_GRF.update({(leg): sym.Matrix([[GRF['x',leg]],[GRF['y',leg]]])})
    lamda_connect.update({('left',leg): sym.Matrix([[-lamda['x',leg]],[-lamda['y',leg]]])})
    lamda_connect.update({('right',leg): sym.Matrix([[lamda['x',leg]],[lamda['y',leg]]])})
    
    Tau = Tau.col_join(sym.Matrix([[tau['left',leg]],[0],[tau['right',leg]],[0]]))
    
    Q = Q + (Jfoot[leg].transpose()*lamda_GRF[leg]) + (Jbn['left',leg].transpose()*lamda_connect['left',leg]) + (Jbn['right',leg].transpose()*lamda_connect['right',leg])

In [11]:
# Manipulator Equation 
EOM = M*ddq + G + C - Tau - Q 

# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('unsimplified_monopod.db')

In [12]:
EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()

In [13]:
# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('simplified_monopod.db')